# **Visual Information Processing and Management**
---
---

Università degli Studi Milano Bicocca \
CdLM Informatica — A.A 2025/2026

---
---

### **Componenti del gruppo:**
— Oleksandra Golub (856706) \
— Andrea Spagnolo (879254)




## **Librerie**


In [1]:
import os
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from PIL import Image, ImageOps
from dataclasses import dataclass
from typing import List, Tuple

from sklearn.linear_model import LogisticRegression, RidgeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier

from sklearn.metrics import f1_score, accuracy_score, confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.base import clone
from skimage import exposure
from skimage.restoration import denoise_bilateral
from sklearn.decomposition import PCA

from skimage.feature import hog, local_binary_pattern
from sklearn.svm import LinearSVC
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

from torchvision import datasets

## **Configurazione + Device**

In [2]:
@dataclass
class Config:
    # paths
    data_dir = "/kaggle/input/datasets/andreaspagnolo/visual-exam-dataset/visual_dataset"
    #data_dir: str = "/kaggle/input/visual-exam-dataset/visual_dataset"
    fit_dir  = "/kaggle/input/datasets/andreaspagnolo/visual-exam-dataset/visual_dataset/valid"
    #fit_dir: str = "/kaggle/input/visual-exam-dataset/visual_dataset/valid"
    test_dir = "/kaggle/input/datasets/andreaspagnolo/visual-exam-dataset/visual_dataset/test_degradato"
    #test_dir: str = "/kaggle/input/visual-exam-dataset/visual_dataset/test_degradato"
    cache_dir = "/kaggle/working/feat_cache"
    
    img_size = 128   # per HOG spesso 128 va benissimo

    # ===== preprocessing =====
    use_preprocess = True
    use_grayworld = False          # per degradato spesso può fare danni → default OFF
    use_autocontrast = False       # lo sostituiamo con CLAHE
    use_clahe = True
    clahe_clip_limit = 0.01        # 0.01–0.03 di solito ok; aumenta se immagini “piatte”
    use_denoise = True
    denoise_sigma_color = 0.05     # leggero
    denoise_sigma_spatial = 2      # leggero

    # ===== HOG params (default, poi li tuneremo) =====
    hog_orientations = 9
    hog_pixels_per_cell = (8, 8)
    hog_cells_per_block = (2, 2)
    hog_transform_sqrt = True

    # ===== tuning =====
    test_size_internal = 0.2
    random_state = 42
    C_grid = (0.1, 0.3, 1.0, 3.0, 10.0)
    max_iter = 50000

    # ===== PCA =====
    use_pca = True
    pca_components_grid = (256, 512, 1024)  # filtreremo automaticamente quelli non validi

def sanity_check_paths(cfg: Config):
    for p in [cfg.data_dir, cfg.fit_dir, cfg.test_dir]:
        if not os.path.exists(p):
            raise FileNotFoundError(f"Path not found: {p}")
    print("OK paths")
    print("fit_dir:", cfg.fit_dir)
    print("test_dir:", cfg.test_dir)

In [3]:

def ensure_cache_dir(cfg):
    os.makedirs(cfg.cache_dir, exist_ok=True)

def feature_signature(cfg, mode: str) -> str:
    return (
        f"{mode}"
        f"_img{cfg.img_size}"
        f"_pre{int(cfg.use_preprocess)}"
        f"_clahe{int(cfg.use_clahe)}c{cfg.clahe_clip_limit}"
        f"_den{int(cfg.use_denoise)}sc{cfg.denoise_sigma_color}ss{cfg.denoise_sigma_spatial}"
        f"_hogO{cfg.hog_orientations}"
        f"_hogpc{cfg.hog_pixels_per_cell[0]}x{cfg.hog_pixels_per_cell[1]}"
        f"_hogcb{cfg.hog_cells_per_block[0]}x{cfg.hog_cells_per_block[1]}"
        f"_hogsqrt{int(cfg.hog_transform_sqrt)}"
    )


def cache_paths(cfg, mode: str):
    sig = feature_signature(cfg, mode)
    fit_path  = os.path.join(cfg.cache_dir, f"Xy_fit_{sig}.npz")
    test_path = os.path.join(cfg.cache_dir, f"Xy_test_{sig}.npz")
    return fit_path, test_path

def load_xy_npz(path: str):
    data = np.load(path)
    return data["X"], data["y"]

def save_xy_npz(path: str, X: np.ndarray, y: np.ndarray):
    np.savez_compressed(path, X=X, y=y)

## **Preprocessing**

In [4]:
class GrayWorldWB:
    def __call__(self, img: Image.Image) -> Image.Image:
        arr = np.asarray(img).astype(np.float32)
        mean = arr.mean(axis=(0, 1), keepdims=True)
        arr = arr / (mean + 1e-6)
        arr = arr / (arr.max() + 1e-6)
        return Image.fromarray((arr * 255).astype(np.uint8))

class AutoContrast:
    def __call__(self, img: Image.Image) -> Image.Image:
        return ImageOps.autocontrast(img, cutoff=1)

def preprocess_pil(img: Image.Image, cfg: Config) -> Image.Image:
    if not cfg.use_preprocess:
        return img
    if cfg.use_grayworld:
        img = GrayWorldWB()(img)
    if cfg.use_autocontrast:
        img = AutoContrast()(img)
    return img


def preprocess_gray(img_gray: np.ndarray, cfg: Config) -> np.ndarray:
    """
    img_gray: uint8 [0..255]
    ritorna uint8 [0..255]
    """
    if not cfg.use_preprocess:
        return img_gray

    x = img_gray.astype(np.float32) / 255.0

    # Denoise leggero (bilateral) – aiuta su rumore e degradazione
    if cfg.use_denoise:
        x = denoise_bilateral(
            x,
            sigma_color=cfg.denoise_sigma_color,
            sigma_spatial=cfg.denoise_sigma_spatial,
            channel_axis=None
        )

    # CLAHE (adaptive histogram equalization)
    if cfg.use_clahe:
        x = exposure.equalize_adapthist(x, clip_limit=cfg.clahe_clip_limit)

    x = np.clip(x, 0, 1)
    return (x * 255).astype(np.uint8)


## **Feature Extraction**

In [5]:
def feat_hog(path: str, cfg: Config) -> np.ndarray:
    img = Image.open(path).convert("RGB")
    img = img.resize((cfg.img_size, cfg.img_size))
    img_gray = np.array(img.convert("L"))

    img_gray = preprocess_gray(img_gray, cfg)

    return hog(
        img_gray,
        orientations=cfg.hog_orientations,
        pixels_per_cell=cfg.hog_pixels_per_cell,
        cells_per_block=cfg.hog_cells_per_block,
        block_norm="L2-Hys",
        transform_sqrt=cfg.hog_transform_sqrt,
        feature_vector=True
    ).astype(np.float32)

def feat_lbp(path: str, cfg: Config, P: int = 8, R: int = 1) -> np.ndarray:
    img = Image.open(path).convert("RGB")
    img = preprocess_pil(img, cfg)
    img = img.resize((cfg.img_size, cfg.img_size))
    img_gray = np.array(img.convert("L"))

    lbp = local_binary_pattern(img_gray, P=P, R=R, method="uniform")
    n_bins = P + 2
    hist, _ = np.histogram(lbp.ravel(), bins=np.arange(0, n_bins + 1), range=(0, n_bins))
    hist = hist.astype(np.float32)
    hist /= (hist.sum() + 1e-8)
    return hist

def feat_hog_lbp(path: str, cfg: Config) -> np.ndarray:
    return np.concatenate([feat_hog(path, cfg), feat_lbp(path, cfg)], axis=0)

## **Dataset**

In [6]:
def build_xy(folder: datasets.ImageFolder, cfg: Config, mode: str) -> Tuple[np.ndarray, np.ndarray]:
    X, y = [], []
    for path, label in folder.samples:
        if mode == "hog":
            X.append(feat_hog(path, cfg))
        elif mode == "lbp":
            X.append(feat_lbp(path, cfg))
        elif mode == "hog+lbp":
            X.append(feat_hog_lbp(path, cfg))
        else:
            raise ValueError("mode must be: 'hog', 'lbp', 'hog+lbp'")
        y.append(label)
    return np.asarray(X), np.asarray(y)

## **Evaluation su test + Matrice di confusione**

In [7]:
def plot_confusion_matrix(cm: np.ndarray, title: str = "Confusion Matrix (normalized)"):
    cm_norm = cm.astype(np.float32) / np.maximum(cm.sum(axis=1, keepdims=True), 1)

    plt.figure(figsize=(18, 16))
    sns.heatmap(cm_norm, cmap="viridis", xticklabels=False, yticklabels=False)
    plt.title(title)
    plt.xlabel("Predicted label")
    plt.ylabel("True label")
    plt.tight_layout()
    plt.show()


def print_top_confusions(cm: np.ndarray, class_names: List[str], top_k: int = 100):
    cm_no_diag = cm.copy()
    np.fill_diagonal(cm_no_diag, 0)

    pairs = []
    for i in range(len(class_names)):
        for j in range(len(class_names)):
            if i != j and cm_no_diag[i, j] > 0:
                pairs.append((i, j, cm_no_diag[i, j]))

    pairs_sorted = sorted(pairs, key=lambda x: x[2], reverse=True)

    print(f"\nTop {top_k} coppie più confuse:\n")
    for i, j, value in pairs_sorted[:top_k]:
        print(f"{class_names[i]} → {class_names[j]}  | {value} errori")

def tune_and_test_model(X_fit, y_fit, X_test, y_test, cfg: Config, model_name: str, base_pipe: Pipeline, param_grid: dict):
    # split interno sul VALID
    X_tr, X_val, y_tr, y_val = train_test_split(
        X_fit, y_fit,
        test_size=cfg.test_size_internal,
        random_state=cfg.random_state,
        stratify=y_fit
    )

    best_params = None
    best_f1 = -1.0
    best_pipe = None

    # grid search manuale (semplice e controllabile)
    # param_grid: dict tipo {"svm__C":[...], "knn__n_neighbors":[...]}
    keys = list(param_grid.keys())
    values = list(param_grid.values())

    def product(lst):
        if not lst:
            return [()]
        out = [()]
        for v in lst:
            out = [x + (y,) for x in out for y in v]
        return out

    for combo in product(values):
        pipe = clone(base_pipe)
        params = {k: combo[i] for i, k in enumerate(keys)}
        pipe.set_params(**params)

        pipe.fit(X_tr, y_tr)
        y_val_pred = pipe.predict(X_val)

        f1 = f1_score(y_val, y_val_pred, average="macro", zero_division=0)
        if f1 > best_f1:
            best_f1 = f1
            best_params = params
            best_pipe = pipe

    # fit finale su TUTTO il VALID
    final_pipe = clone(base_pipe)
    if best_params is not None:
        final_pipe.set_params(**best_params)

    final_pipe.fit(X_fit, y_fit)
    y_pred = final_pipe.predict(X_test)

    acc = accuracy_score(y_test, y_pred)
    f1  = f1_score(y_test, y_pred, average="macro", zero_division=0)
    cm  = confusion_matrix(y_test, y_pred)

    return best_params, best_f1, acc, f1, cm, final_pipe


def get_or_build_xy(folder, cfg, mode: str, split_name: str):
    """
    split_name: "fit" o "test"
    """
    ensure_cache_dir(cfg)
    fit_path, test_path = cache_paths(cfg, mode)
    path = fit_path if split_name == "fit" else test_path

    if os.path.exists(path):
        print(f"[CACHE HIT] {split_name} {mode} -> {path}")
        return load_xy_npz(path)

    print(f"[CACHE MISS] calcolo feature per {split_name} {mode}...")
    X, y = build_xy(folder, cfg, mode)   # <-- la tua funzione build_xy
    save_xy_npz(path, X, y)
    print(f"[CACHE SAVE] {split_name} {mode} -> {path}")
    return X, y


In [8]:
def get_models(cfg: Config, pca_n_components_list, use_pca_now: bool):

    models = {}

    # blocco PCA opzionale
    pca_step = []
    if use_pca_now:
        pca_step = [("pca", PCA(n_components=pca_n_components_list[0], svd_solver="randomized", random_state=cfg.random_state))]

    # 1) LinearSVC
    models["LinearSVC"] = (
        Pipeline([("scaler", StandardScaler())] + pca_step + [("clf", LinearSVC(max_iter=cfg.max_iter))]),
        {
            "clf__C": list(cfg.C_grid),
            **({"pca__n_components": pca_n_components_list} if use_pca_now else {})
        }
    )

    # 2) Logistic Regression
    models["LogReg"] = (
        Pipeline([("scaler", StandardScaler())] + pca_step + [("clf", LogisticRegression(max_iter=5000, n_jobs=-1))]),
        {
            "clf__C": list(cfg.C_grid),
            **({"pca__n_components": pca_n_components_list} if use_pca_now else {})
        }
    )

    # 3) RidgeClassifier (molto veloce)
   # models["Ridge"] = (
   #     Pipeline([("scaler", StandardScaler()),
   #               ("clf", RidgeClassifier())]),
   #     {"clf__alpha": [0.1, 1.0, 3.0, 10.0]}
   # )

    # 4) KNN (scale obbligatoria)
  #  models["KNN"] = (
   #     Pipeline([("scaler", StandardScaler()),
   #               ("clf", KNeighborsClassifier())]),
   #     {"clf__n_neighbors": [3, 5, 11, 21], "clf__weights": ["uniform", "distance"]}
   # )

    # 5) RandomForest (NON serve scaler, ma non fa male tenerlo fuori)
    # Qui faccio pipeline senza scaler
  #  models["RF"] = (
  #      Pipeline([("clf", RandomForestClassifier(random_state=cfg.random_state, n_jobs=-1))]),
   #     {"clf__n_estimators": [200, 500], "clf__max_depth": [None, 20, 40]}
   # )

    # 6) Naive Bayes (GaussianNB) – spesso ok su feature dense
  #  models["GNB"] = (
  #      Pipeline([("scaler", StandardScaler()),
  #                ("clf", GaussianNB())]),
  #      {}  # niente iperparametri principali
  #  )

    return models


In [9]:
hog_grid = [
    dict(hog_orientations=9,  hog_pixels_per_cell=(8,8),  hog_cells_per_block=(2,2), hog_transform_sqrt=True),
    dict(hog_orientations=12, hog_pixels_per_cell=(8,8),  hog_cells_per_block=(2,2), hog_transform_sqrt=True),
    dict(hog_orientations=9,  hog_pixels_per_cell=(16,16),hog_cells_per_block=(2,2), hog_transform_sqrt=True),
    dict(hog_orientations=12, hog_pixels_per_cell=(16,16),hog_cells_per_block=(2,2), hog_transform_sqrt=True),
    dict(hog_orientations=9,  hog_pixels_per_cell=(16,16),hog_cells_per_block=(3,3), hog_transform_sqrt=True),
]

In [10]:
pre_pca_grid = [
    {"use_preprocess": False, "use_pca": False, "tag": "PRE0_PCA0"},
    {"use_preprocess": False, "use_pca": True,  "tag": "PRE0_PCA1"},
    {"use_preprocess": True,  "use_pca": False, "tag": "PRE1_PCA0"},
    {"use_preprocess": True,  "use_pca": True,  "tag": "PRE1_PCA1"},
]


## **Main**

In [11]:
cfg = Config()
np.random.seed(cfg.random_state)
sanity_check_paths(cfg)

OK paths
fit_dir: /kaggle/input/datasets/andreaspagnolo/visual-exam-dataset/visual_dataset/valid
test_dir: /kaggle/input/datasets/andreaspagnolo/visual-exam-dataset/visual_dataset/test_degradato


In [12]:
fit_ds  = datasets.ImageFolder(cfg.fit_dir)   # VALID come training del task3
test_ds = datasets.ImageFolder(cfg.test_dir)

print("Fit classes:", fit_ds.classes)
print("Test classes:", test_ds.classes)
assert fit_ds.classes == test_ds.classes, "ATTENZIONE: class order diverso tra fit e test!"

class_names = fit_ds.classes

Fit classes: ['air hockey', 'ampute football', 'archery', 'arm wrestling', 'axe throwing', 'balance beam', 'barell racing', 'baseball', 'basketball', 'baton twirling', 'bike polo', 'billiards', 'bmx', 'bobsled', 'bowling', 'boxing', 'bull riding', 'bungee jumping', 'canoe slamon', 'cheerleading', 'chuckwagon racing', 'cricket', 'croquet', 'curling', 'disc golf', 'fencing', 'field hockey', 'figure skating men', 'figure skating pairs', 'figure skating women', 'fly fishing', 'football', 'formula 1 racing', 'frisbee', 'gaga', 'giant slalom', 'golf', 'hammer throw', 'hang gliding', 'harness racing', 'high jump', 'hockey', 'horse jumping', 'horse racing', 'horseshoe pitching', 'hurdles', 'hydroplane racing', 'ice climbing', 'ice yachting', 'jai alai', 'javelin', 'jousting', 'judo', 'lacrosse', 'log rolling', 'luge', 'motorcycle racing', 'mushing', 'nascar racing', 'olympic wrestling', 'parallel bar', 'pole climbing', 'pole dancing', 'pole vault', 'polo', 'pommel horse', 'rings', 'rock climbi

In [13]:
mode = "hog"
best_overall = None  # (test_f1, info_dict)

all_results = {}
cms = {}

#modes = ["hog", "lbp", "hog+lbp"]

for setting in pre_pca_grid:
    cfg.use_preprocess = setting["use_preprocess"]
    cfg.use_pca = setting["use_pca"]

    # IMPORTANTE: resetta cache per evitare confusione tra settaggi vecchi
    # (non è obbligatorio, perché la signature include pre/pca, ma è più pulito)
    # cfg.cache_dir = f"/kaggle/working/feat_cache_{setting['tag']}"
    # ensure_cache_dir(cfg)

    print("\n############################################")
    print("SETTING:", setting["tag"], "| preprocess:", cfg.use_preprocess, "| pca:", cfg.use_pca)
    print("############################################")

    for hg in hog_grid:
        cfg.hog_orientations = hg["hog_orientations"]
        cfg.hog_pixels_per_cell = hg["hog_pixels_per_cell"]
        cfg.hog_cells_per_block = hg["hog_cells_per_block"]
        cfg.hog_transform_sqrt = hg["hog_transform_sqrt"]
    
        print("\n====================")
        print("HOG CONFIG:", hg)
        print("====================")
    
        X_fit, y_fit   = get_or_build_xy(fit_ds,  cfg, mode, split_name="fit")
        X_test, y_test = get_or_build_xy(test_ds, cfg, mode, split_name="test")
    
        use_pca_now = cfg.use_pca

        max_n = min(X_fit.shape[0] - 1, X_fit.shape[1])
        pca_list = [n for n in cfg.pca_components_grid if n <= max_n and n >= 10]
        
        if use_pca_now and (len(pca_list) == 0):
            print(f"[WARN] Nessun n_components valido per PCA (max {max_n}). Disabilito PCA solo per questo giro.")
            use_pca_now = False
            pca_list = []

    
        models = get_models(cfg, pca_list if pca_list else [None], use_pca_now)
    
        for model_name, (pipe, grid) in models.items():
            if (not use_pca_now) and ("pca__n_components" in grid):
                grid = {k: v for k, v in grid.items() if k != "pca__n_components"}
    
            best_params, best_val_f1, test_acc, test_f1, cm, _ = tune_and_test_model(
                X_fit, y_fit, X_test, y_test, cfg, model_name, pipe, grid
            )
    
            print(f"\n--- {model_name} ---")
            print("Best params:", best_params)
            print(f"Val Macro-F1: {best_val_f1:.4f}")
            print(f"TEST Acc:     {test_acc:.4f}")
            print(f"TEST Macro-F1:{test_f1:.4f}")
    
            key = (setting["tag"], str(hg), model_name)
            all_results[key] = (best_params, best_val_f1, test_acc, test_f1)
            cms[key] = cm

    
            # salva best overall (come avevi già)
            info = {
                "tag": setting["tag"],
                "hog": hg,
                "model": model_name,
                "best_params": best_params,
                "val_f1": best_val_f1,
                "test_acc": test_acc,
                "test_f1": test_f1
            }

            if (best_overall is None) or (test_f1 > best_overall[0]):
                best_overall = (test_f1, info)

print("\n=== BEST OVERALL (by TEST Macro-F1) ===")
print(best_overall[1])


############################################
SETTING: PRE0_PCA0 | preprocess: False | pca: False
############################################

HOG CONFIG: {'hog_orientations': 9, 'hog_pixels_per_cell': (8, 8), 'hog_cells_per_block': (2, 2), 'hog_transform_sqrt': True}
[CACHE MISS] calcolo feature per fit hog...
[CACHE SAVE] fit hog -> /kaggle/working/feat_cache/Xy_fit_hog_img128_pre0_clahe1c0.01_den1sc0.05ss2_hogO9_hogpc8x8_hogcb2x2_hogsqrt1.npz
[CACHE MISS] calcolo feature per test hog...
[CACHE SAVE] test hog -> /kaggle/working/feat_cache/Xy_test_hog_img128_pre0_clahe1c0.01_den1sc0.05ss2_hogO9_hogpc8x8_hogcb2x2_hogsqrt1.npz

--- LinearSVC ---
Best params: {'clf__C': 0.1}
Val Macro-F1: 0.1223
TEST Acc:     0.1120
TEST Macro-F1:0.1064

--- LogReg ---
Best params: {'clf__C': 0.3}
Val Macro-F1: 0.1287
TEST Acc:     0.1480
TEST Macro-F1:0.1367

HOG CONFIG: {'hog_orientations': 12, 'hog_pixels_per_cell': (8, 8), 'hog_cells_per_block': (2, 2), 'hog_transform_sqrt': True}
[CACHE MISS] calco

In [14]:
print("\n=== RIASSUNTO FINALE ===")
for (tag, hog_conf, model_name), (best_params, best_val_f1, test_acc, test_f1) in all_results.items():
    print(f"{tag} | {hog_conf} | {model_name:8} | ValF1={best_val_f1:.4f} | TestAcc={test_acc:.4f} | TestF1={test_f1:.4f} | {best_params}")



=== RIASSUNTO FINALE ===
PRE0_PCA0 | {'hog_orientations': 9, 'hog_pixels_per_cell': (8, 8), 'hog_cells_per_block': (2, 2), 'hog_transform_sqrt': True} | LinearSVC | ValF1=0.1223 | TestAcc=0.1120 | TestF1=0.1064 | {'clf__C': 0.1}
PRE0_PCA0 | {'hog_orientations': 9, 'hog_pixels_per_cell': (8, 8), 'hog_cells_per_block': (2, 2), 'hog_transform_sqrt': True} | LogReg   | ValF1=0.1287 | TestAcc=0.1480 | TestF1=0.1367 | {'clf__C': 0.3}
PRE0_PCA0 | {'hog_orientations': 12, 'hog_pixels_per_cell': (8, 8), 'hog_cells_per_block': (2, 2), 'hog_transform_sqrt': True} | LinearSVC | ValF1=0.1123 | TestAcc=0.1100 | TestF1=0.1054 | {'clf__C': 0.1}
PRE0_PCA0 | {'hog_orientations': 12, 'hog_pixels_per_cell': (8, 8), 'hog_cells_per_block': (2, 2), 'hog_transform_sqrt': True} | LogReg   | ValF1=0.1539 | TestAcc=0.1520 | TestF1=0.1444 | {'clf__C': 1.0}
PRE0_PCA0 | {'hog_orientations': 9, 'hog_pixels_per_cell': (16, 16), 'hog_cells_per_block': (2, 2), 'hog_transform_sqrt': True} | LinearSVC | ValF1=0.0923 | T

In [15]:
for (tag, hog_conf, model_name), cm in cms.items():
    print(f"\n=== Top confusions: {tag} | {hog_conf} + {model_name} ===")
    print_top_confusions(cm, class_names, top_k=30)



=== Top confusions: PRE0_PCA0 | {'hog_orientations': 9, 'hog_pixels_per_cell': (8, 8), 'hog_cells_per_block': (2, 2), 'hog_transform_sqrt': True} + LinearSVC ===

Top 30 coppie più confuse:

baton twirling → trapeze  | 2 errori
disc golf → rings  | 2 errori
figure skating men → figure skating pairs  | 2 errori
hammer throw → hurdles  | 2 errori
horse jumping → curling  | 2 errori
olympic wrestling → curling  | 2 errori
roller derby → cricket  | 2 errori
sailboat racing → ice yachting  | 2 errori
snow boarding → trapeze  | 2 errori
sumo wrestling → horseshoe pitching  | 2 errori
table tennis → trapeze  | 2 errori
tennis → hang gliding  | 2 errori
ultimate → bike polo  | 2 errori
air hockey → bmx  | 1 errori
air hockey → curling  | 1 errori
air hockey → hockey  | 1 errori
air hockey → nascar racing  | 1 errori
ampute football → axe throwing  | 1 errori
ampute football → olympic wrestling  | 1 errori
ampute football → speed skating  | 1 errori
ampute football → uneven bars  | 1 errori
ar